In [7]:
class BlackjackAgent():
    
#     blev = BlackjackEnv(natural=False)
#     p1 = Player(player_name='C1')
#     blev.add_player(p1)
    def reset_env(self):
        #print('here')
        pkev = BlackjackEnv(natural=False)
        p1 = Player(player_name='C1')
        p2 = Player(player_name='P1')
        self.p1 = p1
        pkev.add_player(p1)
        #print(p1)
        return pkev
    
    def __init__(self):
        self.env = self.reset_env()
        self.Q = np.zeros((34,2))
        
    def epsilon_greed(self, epsilon, s):
        #ac = [self.env.get_valid_actions(self.me)]
        ac = [0,1]
        if np.random.rand()<epsilon:
            return np.random.randint(len(ac))
        else:
            #print("agd : ",s)
            #print("fld:",s['player_sum_card'])
            #print("agd : ",ac)
            return np.argmax(self.Q[s['player_sum_card']][ac])
    
    def train(self,**params):
        
        # parameters
        gamma = params.pop('gamma', 0.99)
        alpha = params.pop('alpha', 0.1)
        epsilon = params.pop('epsilon', 0.1)
        maxiter = params.pop('maxiter', 1000) 
        sarsa = params.pop('sarsa',False)
        edecay = params.pop('edecay',None)
        import time
        rtrace = []
        for i in range(maxiter):
            if i == 20000:
                #print(self.Q)
                time.sleep(5)
            self.env = self.reset_env()
            if edecay is not None:
                epsilon = max(0.1,epsilon*edecay)
            #print('*'*50,i+1,'*'*50)
            #Check if you have enough money left in the account to bet, if not break and start a new game
            valid_bet_flag = self.env.get_valid_bet_amount(self.p1.player_name)
            if valid_bet_flag['is_round_done']:
                #print("You are out of money ! Will go to next game !!")
                break
            bet_amount = valid_bet_flag['valid_bet_amount']
        
            #Get the observations i.e state and use it to decide how much you want to bet
            round_obs = self.env.init_round(self.p1.player_name)
            #print("Initial state: {}".format(round_obs))  
        
            # If you have just $1 left in the bank, you have no choice but to bet that amount 
            # and hopefully leave it to the fate for your agent to win and continue playing or just
            # start a new game
            if len(bet_amount) == 1 and bet_amount[0] == 1:
                #print("You can only bet 1. So betting only 1.")
                random_bet = 1
            else:
                random_bet = np.random.choice(bet_amount)
                #print("Random Init Bet: {}".format(random_bet))
            self.env.bet_money(self.p1.player_name, random_bet)
        
            random_action = np.random.randint(0, 2)
            #print("Action taken: {}".format(random_action))
            
            #Use the selected action to actually take the action in env by calling step
            res = self.env.step(self.p1.player_name, random_action)
            #print("State after 1st action: {}".format(round_obs))
            #rewards = []
            #res = {'is_round_done':False}
            while not res['is_round_done']:
                #print('-'*50)
                #print(self.env.get_current_state())
                f = 0
                s = self.env.get_player_obs(self.p1.player_name)
                a = self.epsilon_greed(epsilon,s)
                #va = [self.env.get_valid_actions(self.me)]
                va = [0,1]
                res = self.env.step(self.p1.player_name, a)
                #res = self.env.player_play(self.me,va[a])
                #end = res[-1]
                r = res['reward']
                rewards.append(r)
                #print("1:",rewards)
                if not res['is_round_done']:
                    f = 1
                    s1 = self.env.get_player_obs(self.p1.player_name)
                    a1 = self.epsilon_greed(epsilon,s1)
                    #va1 = self.env.get_valid_actions(self.me)
                    res = self.env.step(self.p1.player_name, a1)
                    r1 = res['reward']
                    rewards.append(r1)
                    #print("2:",rewards)
                s2 = self.env.get_player_obs(self.p1.player_name)
                #print("S => ",s)
                #print("S2 => ",s2)
#                 print("s -> " ,s)
#                 print("s1 -> ",s1)
                    # if f:
                    # self.Q[s1,a1]
                #print("va=> ",va)
                #print("a=> ",a)
                self.Q[s['player_sum_card']][a] += alpha*(r + gamma*np.max(self.Q[s2['player_sum_card']][va]) - 
                                      self.Q[s['player_sum_card']][a])  
                
            self.env = self.reset_env()
            rtrace.append(np.sum(rewards))
            #print(rtrace)
        return rtrace,Q

In [ ]:
import socket
import os
import subprocess
import ast
import numpy as np
#import BlackjackAgent
player = 'supreeth' #enter unique name for your game
s = socket.socket()
host = '10.216.25.34' # change ip addres to ip adress of your computer or use 'localhost' to practice
port = 9999

s.connect((host, port))

while True:
    data = s.recv(1024)
    if data.decode("utf-8") == 'send':
        s.send(str.encode( player))
        client_response = str(s.recv(20480), "utf-8")
        print(client_response, end="")
        break
while True:
    data = s.recv(1024)
    if data.decode("utf-8") == 'sendbet':
        ro=str(s.recv(20480), "utf-8")
        ro=ast.literal_eval(ro)
        print(ro)
# The above observation are stored in dict format. To access specific variables use syntax as 
# ro['state']['player_info']['player_total_balance']
# send whichever variable information stored in ro to your agent to help make the decision
#### Look at the above observations stored in variable ro as dictionary and store bet amount in variable bet#####
        bet=np.random.choice(10)
        s.send(str.encode( str(bet)))
        
    if data.decode("utf-8") == 'sendaction':
        ro=str(s.recv(20480), "utf-8")
        ro=ast.literal_eval(ro)
        print(ro)
        #follow same procedure as abov to send observations to your agent
        ###### Look at the above observations and action in variable action#####
        
        action = np.random.randint(0, 2)
        #proplayer1 = BlackjackAgent()
        #proplayer1.train(maxiter = 120,edecay=0.99,epsilon=0.2,gamma=0.8)
        #action = np.argmax(self.Q[s['player_sum_card']][valid_ac])
        #print("action ",action)
        s.send(str.encode( str(action)))
    if data.decode("utf-8") == 'gameover':
        print("Game over wait for others to play and wait for results")
        
        
    